In [98]:
import scipy.io
import torch
import importlib
import matplotlib.pyplot as plt
import numpy as np

In [99]:
import src.HyperParameters as hp 
import src.data as dt 
import src.model3 as Model 
import src.train as trainer

importlib.reload(hp)
importlib.reload(dt)
importlib.reload(Model)
importlib.reload(trainer)

<module 'src.train' from '/Users/ramtarun/Desktop/Cambridge/Friction-Factor-Estimation-PINN/src/train.py'>

In [100]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')

In [101]:
PINN_model = Model.PINN(hp.input_size, hp.output_size, hp.hidden_sizes, hp.activations)
PINN_model.train()

PINN(
  (loss_function): MSELoss()
  (rnn): PhyGRU(
    (activation): ModuleList(
      (0): ReLU()
      (1): Tanh()
      (2): ReLU()
      (3): Tanh()
      (4): ReLU()
      (5): Tanh()
      (6): ReLU()
      (7): Sigmoid()
    )
    (gru): GRU(2, 6, batch_first=True)
    (hidden_layers): ModuleList(
      (0): Linear(in_features=6, out_features=6, bias=True)
      (1): Linear(in_features=6, out_features=6, bias=True)
      (2): Linear(in_features=6, out_features=6, bias=True)
      (3): Linear(in_features=6, out_features=6, bias=True)
      (4): Linear(in_features=6, out_features=6, bias=True)
      (5): Linear(in_features=6, out_features=6, bias=True)
      (6): Linear(in_features=6, out_features=6, bias=True)
      (7): Linear(in_features=6, out_features=6, bias=True)
    )
    (output_layer): Linear(in_features=6, out_features=4, bias=True)
  )
)

In [102]:
params = list(PINN_model.parameters())
# optimizer = torch.optim.Adam([{'params' : params[1::]},{'params' : params[-1], 'lr': hp.ff_learning_rate}], lr = hp.learning_rate, amsgrad = True)   
optimizer = torch.optim.Adam(params=params, lr = hp.learning_rate, amsgrad = True)   
# optimizer = torch.optim.LBFGS(params, hp.ff_learning_rate, 
#                               max_iter = hp.epochs, 
#                               max_eval = None, 
#                               tolerance_grad = 1e-11, 
#                               tolerance_change = 1e-11, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')


In [103]:
fval = [0.1  , 0.06 , 0.04, 0.01]

# for f in fval:
inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=0.01)
N = inputs.shape[1]
train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)
# ### Model 3
# batch_size = 32
# sequence_length = len(inputs) // batch_size
# # Reshape the input tensors within the DataLoader
# train_loader = []
# for batch in Train_loader:
#     input_data = batch.view(batch_size, sequence_length, 1)
#     train_loader.append(input_data)

train_loss, val_loss, f_train, f_test, f_dist = trainer.train(train_loader, val_loader, hp.epochs, optimizer, PINN_model, N)
with torch.no_grad():
    plt.figure() 
    plt.plot(train_loss.keys(), train_loss.values(), 'r-', label='Training Loss')
    plt.plot(val_loss.keys(), val_loss.values(), 'g-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
    plt.plot(f_train.keys(), f_train.values(), 'r-', label='Training Friction Factor')
    plt.plot(f_test.keys(), f_test.values(), 'g-', label='Validation Friction Factor')
    plt.xlabel('Epochs')
    plt.ylabel('Friction Factor')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
    ff_distribution = {}
    for key_tensor, value_tensor in f_dist.items():
    # Convert the tensors to numpy arrays
        key_array = np.array(key_tensor.detach())
        value_array = np.array(value_tensor.detach())
        # Unpack the elements from the tensors
        for key_elem, value_elem in zip(key_array, value_array):
        # Add the unpacked elements to the new dictionary
            ff_distribution[key_elem] = value_elem
        
        
ff_distribution = dict(sorted(ff_distribution.items()))
plt.plot(ff_distribution.keys(), ff_distribution.values(), label='Friction Factor Variation along the Nozzle')
plt.ylabel('Friction Factor')
plt.xlabel('Eta')
plt.ylim([0,1]) 
plt.legend()
plt.grid()
plt.show()


Epoch 75/7500 - Train Loss: 10.712062 Val Loss: 0.508682 f_train: -0.4848 f_test: -0.4848
Epoch 150/7500 - Train Loss: 3.404699 Val Loss: 0.283232 f_train: -0.4848 f_test: -0.4848
Epoch 225/7500 - Train Loss: 1.150948 Val Loss: 0.159419 f_train: -0.4848 f_test: -0.4848
Epoch 300/7500 - Train Loss: 0.443331 Val Loss: 0.092198 f_train: -0.4848 f_test: -0.4848
Epoch 375/7500 - Train Loss: 0.220936 Val Loss: 0.060958 f_train: -0.4848 f_test: -0.4848
Epoch 450/7500 - Train Loss: 0.150507 Val Loss: 0.054185 f_train: -0.4848 f_test: -0.4848
Epoch 525/7500 - Train Loss: 0.127975 Val Loss: 0.052806 f_train: -0.4848 f_test: -0.4848
Epoch 600/7500 - Train Loss: 0.121062 Val Loss: 0.052525 f_train: -0.4848 f_test: -0.4848
Epoch 675/7500 - Train Loss: 0.118786 Val Loss: 0.052517 f_train: -0.4848 f_test: -0.4848
Epoch 750/7500 - Train Loss: 0.117940 Val Loss: 0.052565 f_train: -0.4848 f_test: -0.4848
Epoch 825/7500 - Train Loss: 0.117689 Val Loss: 0.052610 f_train: -0.4848 f_test: -0.4848
Epoch 900/

In [ ]:
plt.figure()
plt.plot(inputs[:,1], targets)
plt.plot(inputs[:,1],PINN_model.rnn(inputs).detach(), 'k-.')